## New model to include neighborhood effects

v0.1 did not consider neighbors, so estimates were to specific to the source data used for modeling

v0.2 will consider value of wait time of neighbor points 

In [24]:
from datetime import date, datetime
from BorderModel import BorderImpute
from sklearn.ensemble import ExtraTreesRegressor

start = date(2014, 1, 1)
end = date(2014, 3, 1)
imputer = BorderImpute(start, end)

# Get source data with neighbor features
source_query = '''
               select 
                   c.date,
                   year,
                   month,
                   dayofweek,
                   minofday,
                   metric as volume,
                   waittime
               from crossingdata c
               join datefeatures d
                   on d.date = c.date
               join mungedata m 
                   on m.date = c.date
                   and m.crossing_id = c.crossing_id
                   and munger_id = 2
                   and is_waittime = false
               where
                   c.crossing_id=1
                   and valid = 1
                   and volume is not null
                   and (minute = 0 or minute = 30)
                order by c.date
                limit 1000;
               '''
dfsource = imputer.prepare_source(source_query)

In [25]:
dfsource.head()

,year,month,dayofweek,minofday,volume,waittime,lead,lag
date,,,,,,,,
2007-01-01 00:30:00,2007,1,0,30,0.501538,4,0.3,NaN
2007-01-01 01:00:00,2007,1,0,60,1.364060,0,0.4,4.000000
2007-01-01 01:30:00,2007,1,0,90,2.665500,1,0.0,1.714286
2007-01-01 02:00:00,2007,1,0,120,2.100310,0,0.0,1.333333
2007-01-01 02:30:00,2007,1,0,150,1.197660,0,0.0,0.700000


In [26]:
from BorderModel import BorderImpute, xy_laglead

# Run model
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=100, bootstrap=True, oob_score=True)
imputer.build_model(model)

In [27]:
print imputer.model_ll.oob_score_
print imputer.model_lead.oob_score_
print imputer.model_lag.oob_score_

0.847819787716
0.784125570461
0.776267088609


## OOB scores are better than model without lag/lead
Encouraging, but not clear yet whether there is low variance

Need to test against holdout set with waittimes artificially set to zero